In [20]:
import pandas as pd
import numpy as np
import helper
import pnf

# Load df for etfs
etf_dfs = helper.get_sector_etf_dfs()

# Create a Yearly Slice of ETF Data
Using the ETF data loaded from Alpha Vantage, create a slice that only contains data for a specified date range

In [46]:
# Create copy of etf_dfs for backup
master = etf_dfs.copy()
master

,XLB,XLI,XLK,XLP,XLF,XLU,XLE,XLV,XLY
0,Open High Low Close ...,Open High Low Close V...,Open High Low Close V...,Open High Low Close ...,Open High Low Close ...,Open High Low Close ...,Open High Low Close ...,Open High Low Close V...,Open High Low Close ...


In [33]:
# Create a new dataframe for a specified date range
etf_dfs_2017 = master.copy()
start_date = '2017-01-01'
end_date = '2017-12-31'

for symbol in etf_dfs_2017.columns:
    etf_dfs_2017[symbol][0] = etf_dfs_2017[symbol][0][start_date : end_date]

# Create RS Matrix for Sector ETFs
Using the sector ETF data, create an RS matrix to compare the sectors to each other

In [42]:
pnf.run_rs_matrix(list(etf_dfs_2017.columns), etf_dfs_2017)

,x_count,buy_count,total,XLB,XLI,XLK,XLP,XLF,XLU,XLE,XLV,XLY
XLB,4,5,9,NaN,o_buy,x_sell,x_buy,o_sell,x_buy,o_sell,x_buy,o_buy
XLI,3,4,7,x_sell,NaN,o_sell,o_buy,o_sell,x_buy,o_sell,x_buy,o_buy
XLK,5,6,11,x_buy,o_buy,NaN,o_buy,x_sell,x_buy,o_sell,x_buy,x_buy
XLP,2,3,5,o_buy,o_sell,o_sell,NaN,o_sell,x_buy,o_sell,x_buy,o_sell
XLF,5,7,12,x_sell,o_buy,x_buy,x_buy,NaN,x_buy,o_buy,x_buy,o_buy
XLU,0,0,0,o_sell,o_sell,o_sell,o_sell,o_sell,NaN,o_sell,o_sell,o_sell
XLE,8,5,13,x_sell,x_buy,x_buy,x_buy,x_sell,x_buy,NaN,x_buy,x_sell
XLV,1,2,3,o_sell,o_sell,o_sell,o_buy,o_sell,x_buy,o_sell,NaN,o_sell
XLY,6,4,10,x_sell,x_sell,x_sell,x_buy,o_sell,x_buy,o_buy,x_buy,NaN


# TODO:
- Calculate RS Matrix at the end of each month in a year, and aggregate x_count, buy_count, total for the entire year to get an idea of which sector performed the best over the time period
- Create a custom Zipline bundle to hold the sector ETF data. This will let us perform backtest trading using our RS strategy
- Turn output of run_rs_matrix into a nicely formatted table (mostly for visualization practice)


# Create a DataFrame sliced by Month
Create a dataframe that holds a sliced dataframe for each ETF/Month pair

In [55]:
date_ranges = {
    'Jan':('2017-01-01', '2017-01-31'),
    'Feb':('2017-02-01', '2017-02-28'),
    'Mar':('2017-03-01', '2017-03-31'),
    'Apr':('2017-04-01', '2017-04-30'),
    'May':('2017-05-01', '2017-05-31'),
    'Jun':('2017-06-01', '2017-06-30'),
    'Jul':('2017-07-01', '2017-07-31'),
    'Aug':('2017-08-01', '2017-08-31'),
    'Sep':('2017-09-01', '2017-09-30'),
    'Oct':('2017-10-01', '2017-10-31'),
    'Nov':('2017-11-01', '2017-11-30'),
    'Dec':('2017-12-01', '2017-12-31')    
}

# Make a master dataframe for each month in 2017
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
monthly_df = pd.DataFrame(columns=etf_dfs_2017.columns, index=months)

# Iterate through each month
for month in monthly_df.index:
    start_date = date_ranges[month][0]
    end_date = date_ranges[month][1]
    for col in monthly_df.columns:
        monthly_df.at[month, col] = etf_dfs_2017[col][0][start_date : end_date]

# Create RS Charts for Each month
Create an RS chart for each month to be used in yearly aggregations

In [72]:
rs_matrices = {}
for month in months:
    df = pd.DataFrame(monthly_df.loc[month]).transpose()
    rs_matrices[month] = pnf.run_rs_matrix(monthly_df.columns, df)
    

In [81]:
# Iterate through RS Matrices and tally up yearly total counts
yearly_counts = pd.DataFrame(columns=['x_count', 'buy_count', 'total'], index=monthly_df.columns)
yearly_counts = yearly_counts.fillna(0)

for key in list(rs_matrices.keys()):
    yearly_counts['x_count'] += rs_matrices[key]['x_count']
    yearly_counts['buy_count'] += rs_matrices[key]['buy_count']
    yearly_counts['total'] += rs_matrices[key]['total']
    
yearly_counts

,x_count,buy_count,total
XLB,64,7,71
XLI,65,5,70
XLK,69,6,75
XLP,44,1,45
XLF,58,13,71
XLU,49,9,58
XLE,35,9,44
XLV,53,5,58
XLY,68,2,70
